In [1]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.1-py3-none-any.whl size=16265 sha256=f4577c7943a26ebd0b3802f361dbbaa18899ed07b071667bd5687abfecd83d48
  Stored in directory: /root/.cache/pip/wheels/e8/81/8f/29aafd7d671a4e9db4e976d71728870173f0b3f48ccba0bc32
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    try:
      coverpage= soup.find_all('div',class_='img-caption-overlay')
      title=coverpage[0].get_text()
      # print(title)
    except:
      title=''

    
    try:   
      coverpage = soup.find_all('div', class_='entry-header has-post-format')
      time = coverpage[0].find('time')['datetime']
      description=coverpage[0].find('h1').get_text()
      # print(time)
    except:
      time=''
      description=''

    try:
      coverpage = soup.find_all('div', itemprop='articleBody')
      article=''
      for i in coverpage[0].find_all('p'):
        article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('error\n'+url)
  except:
    title=''
    time=''
    description=''
    article=''
    print('url error\n'+url)    

  return [title,description,time,article]

In [8]:
get_articles('https://abi.bo/index.php/noticias/politica/24312-armamento-para-el-golpe-en-bolivia-las-pruebas-se-acumulan-y-se-acercan-a-macri')

['Mauricio Macri y Jeanine Áñez. Foto composición RRSS',
 'Armamento para el golpe en Bolivia: Las pruebas se acumulan y se acercan a Macri',
 '2022-07-12T14:22:01-04:00',
 'La Paz, 12 de julio de 2022 (ABI, información tomada de Página 12). – Un año después de que Página/12 revelara el envío de armas para apoyar el golpe contra Evo Morales, la justicia argentina acumula pruebas del contrabando que salpica a los escalones más altos del gobierno de Mauricio Macri.Los gendarmes citados a prestar testimonio dijeron, en todos los casos, que nunca usaron las balas para entrenarse –como se había intentado instalar para justificar el faltante– y Bolivia confirmó oficialmente que los pertrechos estaban en poder de la Fuerza Aérea y de la Policía local.Mientras tanto, el juez de la causa espera para esta semana el testimonio del comandante del contingente que trasladó los cartuchos de goma y las granadas que terminaron en poder de las fuerzas golpistas.Alejandro Catania –el magistrado a cargo d

In [9]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/agencia/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
output=df['rows'].parallel_apply(get_articles)

df['country']='Argentina'
df['title']=''
df['description']=''
df['time']=''
df['article']=''
for i in range(0,len(df)):
  df.loc[i]['title','description','time','article']=output[i]


df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
name_article='/content/drive/MyDrive/Colab Notebooks/agencia/url_article.csv'
df.to_csv(name_article, index=False)
print(df)


INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


error
https://abi.bo/index.php/noticias/internacional/9523-Las-exportaciones-de-Gazprom-al-extranjero-aumentan-el-23%-en-2021
error
https://abi.bo/index.php/noticias/politica/3243-Cómputo-oficial-al-100%--Jhonny-Llally-es-el-nuevo-Alcalde-de-Potosí
error
https://abi.bo/index.php/noticias/politica/3222-Al-72%-del-cómputo,-el-MAS-se-consolida-como-primera-fuerza-política-en-San-Lorenzo-de-Pando
error
https://abi.bo/index.php/noticias/politica/13225-Diputado-Jauregui-recuerda-a-Camacho-que-el-45%-de-la-población-cruceña-no-respalda-sus-acciones-y-augura-el-fracaso-del-paro-cívico
error
https://abi.bo/index.php/noticias/politica/3214-Al-99,6%-del-cómputo-oficial,-el-MAS-y-R-2025-disputan-la-Alcaldía-de-Sucre-voto-a-voto
error
https://abi.bo/index.php/noticias/politica/5180-Cómputo-al-100%--Santos-Quispe-es-Gobernador-de-La-Paz-y-el-MAS-logra-mayoría-en-la-Asamblea-Legislativa-Departamental
error
https://abi.bo/index.php/noticias/internacional/15277-Venezuela-tiene-más-del-70%-de-vacunados-